# 🎨 SDXL Vintage Illustration Notebook — PRO

Stable Diffusion XL / Turbo / SD 1.5 preconfigured for **vintage ink+watercolor** illustrations (old book style, retro cartoon aesthetic).

### Included
- **Mode Selector** (CPU / GPU basic / GPU optimal via `CONFIG`)
- **Gallery Manager (Flask web UI)** + **Logs** page
- **Text2Img** (default vintage style + custom scene prompt)
- **Img2Img** (photo → vintage illustration)
- **ControlNet (Canny)** for pose/contour consistency
- **Upscale** (x4) for higher resolution
- **Color tone control** (dropdown presets + custom override)
- Saving images + JSON metadata to `/content/outputs` (timestamped)



In [ ]:
## ✅ Tips

- Change **`color_tone`** or **`custom_tone`** any time, then regenerate.
- Use **`seed=None`** for randomization, or set a fixed integer for repeatability.
- For **Img2Img**, tweak **`strength`**:
  - lower = closer to original,
  - higher = stronger style.
- **ControlNet (Canny)** retains silhouette/contours. For different looks, adjust **`canny_low`/`canny_high`**.
- If memory errors occur, run **`free_memory()`** and re-run only the needed loader.


In [ ]:
# @title 🔧 1 Config init
import os
from scripts.config import VARIANTS
from scripts.logger import log_info

CONFIG = {
    "MODE": "GPU_OPTIMAL",       # "CPU", "GPU_BASIC", "GPU_OPTIMAL"
    "MODEL_VARIANT": "SDXL",     # "SDXL", "TURBO", "SD15"
    "OUTPUT_DIR": "/content/outputs",
}
os.makedirs(CONFIG["OUTPUT_DIR"], exist_ok=True)

VARIANT = VARIANTS[CONFIG["MODEL_VARIANT"]]
DEFAULTS = VARIANT["defaults"]
AUTO_UPSCALE = VARIANT["auto_upscale"]

log_info(f"Config initialized: {CONFIG}")


In [ ]:
# @title 📦 2 Server (запуск Flask + Logs)
from scripts.gallery_manager import start_gallery
from scripts.logger import log_info

# Colab → ссылка генерируется через proxyPort (внутри start_gallery)
# Локальный режим → fallback на http://127.0.0.1
start_gallery(CONFIG["OUTPUT_DIR"], port=8000, in_colab=True)
log_info("Gallery server started")



In [ ]:
# @title 📦 3 Install dependencies
!pip -q install diffusers==0.29.0 transformers accelerate xformers safetensors opencv-python-headless ipywidgets flask nest_asyncio

import os, torch
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'
print("Dependencies installed and torch available:", torch.__version__)



In [ ]:
# @title 🔁 4 Imports & utils
from scripts.utils import (
    ts_now, base_name, save_image_and_meta,
    set_seed, free_memory, list_images
)
from scripts.logger import log_info

print(f"Utils loaded. OUTPUT_DIR={CONFIG['OUTPUT_DIR']}")


In [ ]:
# @title 🧠 5 Model loaders
from scripts.loaders import (
    get_txt2img_pipe,
    get_img2img_pipe,
    get_controlnet_pipe,
    get_upscale_pipe,
)

print("Loader functions imported and ready for", CONFIG["MODEL_VARIANT"])



In [ ]:
# @title 🎨 6 Style base & prompt builder

COLOR_TONE_PRESETS = [
    "warm earthy tones",
    "cool tones",
    "sepia",
    "black and white ink",
    "muted colors",
    "vivid watercolor",
]

# Глобальные переменные по умолчанию
color_tone = "warm earthy tones"  # default preset
custom_tone = ""  # если непусто → перекрывает color_tone

STYLE_BASE = (
    "Vintage-style illustration, hand-drawn with ink and watercolor effect on textured parchment background. "
    "Caricatured and symbolic, with {tone}. "
    "Characters in modest or historical attire, expressive faces, bold outlines, soft shading. "
    "Old book illustration style, retro cartoon aesthetic."
)

def build_prompt(scene: str, tone: str | None = None):
    # custom_tone > локальный tone > глобальный color_tone
    tone_final = (custom_tone or tone or color_tone).strip()
    return STYLE_BASE.format(tone=tone_final) + (" " + scene if scene else "")

print("Current color tone:", custom_tone or color_tone)
print("Sample prompt:", build_prompt("A thoughtful man, slightly resembling Elon Musk."))


In [ ]:
# @title 🖼 Text2Img
from scripts.loaders import get_txt2img_pipe
from scripts.utils import save_image_and_meta, ts_now
from scripts.logger import log_info
from scripts.config import VARIANT_MODELS
from PIL import Image

TXT2IMG_MODEL_ID = VARIANT_MODELS[CONFIG["MODEL_VARIANT"]]["txt2img"]

scene = "A thoughtful woman in modest clothing, sitting with hand on chin, pensive pose."
steps = DEFAULTS["txt2img_steps"]
cfg_scale = DEFAULTS["txt2img_cfg"]
height, width = DEFAULTS["img_size"]
seed = 12345
num_images = 1

pipe = get_txt2img_pipe(TXT2IMG_MODEL_ID, DEVICE, DTYPE)
generator = set_seed(seed)
prompt = build_prompt(scene)

log_info(f"Text2Img prompt: {prompt}")

res = pipe(
    prompt,
    height=height,
    width=width,
    num_inference_steps=steps,
    guidance_scale=cfg_scale,
    generator=generator,
)
images = res.images

saved_paths = []
for im in images[:num_images]:
    meta = {
        "model_variant": CONFIG["MODEL_VARIANT"],
        "mode": "text2img",
        "prompt": prompt,
        "scene": scene,
        "seed": seed,
        "steps": steps,
        "cfg_scale": cfg_scale,
        "size": [height, width],
        "color_tone": custom_tone or color_tone,
        "timestamp": ts_now(),
    }
    img_path, meta_path = save_image_and_meta(im, prefix="text2img", meta=meta, output_dir=CONFIG["OUTPUT_DIR"])
    saved_paths.append(img_path)

print("Text2Img done:", saved_paths)


In [ ]:
# @title 🖼 Img2Img
from scripts.loaders import get_img2img_pipe
from scripts.utils import save_image_and_meta, ts_now
from scripts.logger import log_info
from scripts.config import VARIANT_MODELS
from PIL import Image
import os

I2I_MODEL_ID = VARIANT_MODELS[CONFIG["MODEL_VARIANT"]]["i2i"]

input_image_path = ""  # путь или загрузка
strength = 0.5
seed = 12345  # None → случайный

if not input_image_path:
    from google.colab import files
    print("Upload an input image…")
    up = files.upload()
    input_image_path = list(up.keys())[0]

init_img = Image.open(input_image_path).convert("RGB")
pipe_i2i = get_img2img_pipe(I2I_MODEL_ID, DEVICE, DTYPE)
generator = set_seed(seed)

prompt = build_prompt("")
steps_i2i = DEFAULTS["img2img_steps"]
cfg_i2i = DEFAULTS["img2img_cfg"]

log_info(f"Img2Img prompt: {prompt}")

res = pipe_i2i(
    prompt=prompt,
    image=init_img,
    strength=strength,
    num_inference_steps=steps_i2i,
    guidance_scale=cfg_i2i,
    generator=generator,
)
out_img = res.images[0]

meta = {
    "model_variant": CONFIG["MODEL_VARIANT"],
    "mode": "img2img",
    "prompt": prompt,
    "seed": seed,
    "steps": steps_i2i,
    "cfg_scale": cfg_i2i,
    "strength": strength,
    "input": os.path.basename(input_image_path),
    "color_tone": custom_tone or color_tone,
    "timestamp": ts_now(),
}
img_path, meta_path = save_image_and_meta(out_img, prefix="img2img", meta=meta, output_dir=CONFIG["OUTPUT_DIR"])
print("Img2Img done:", img_path)


In [ ]:
# @title 🧭 ControlNet (Canny)
from scripts.loaders import get_controlnet_pipe
from scripts.utils import canny_from_image, save_image_and_meta, ts_now
from scripts.logger import log_info
from scripts.config import VARIANT_MODELS
from PIL import Image
from IPython.display import display

TXT2IMG_MODEL_ID = VARIANT_MODELS[CONFIG["MODEL_VARIANT"]]["txt2img"]
CONTROLNET_CANNY_ID = VARIANT_MODELS[CONFIG["MODEL_VARIANT"]]["controlnet_canny"]

control_input_path = ""  # путь или загрузка
canny_low, canny_high = 100, 200
scene_cn = "Group of legislators sitting around a wooden table, engaged in serious discussion."
seed = 12345  # None → случайный

if not control_input_path:
    from google.colab import files
    print("Upload an image for ControlNet Canny…")
    up = files.upload()
    control_input_path = list(up.keys())[0]

src = Image.open(control_input_path).convert("RGB")
cn_img = canny_from_image(src, canny_low, canny_high)
display(cn_img)

pipe_cn = get_controlnet_pipe(TXT2IMG_MODEL_ID, CONTROLNET_CANNY_ID, DEVICE, DTYPE)
generator = set_seed(seed)

prompt = build_prompt(scene_cn)
steps_cn = DEFAULTS["controlnet_steps"]
cfg_cn = DEFAULTS["controlnet_cfg"]

log_info(f"ControlNet prompt: {prompt}")

out = pipe_cn(
    prompt=prompt,
    image=cn_img,
    num_inference_steps=steps_cn,
    guidance_scale=cfg_cn,
    generator=generator,
)
cn_out = out.images[0]

meta = {
    "model_variant": CONFIG["MODEL_VARIANT"],
    "mode": "controlnet_canny",
    "prompt": prompt,
    "scene": scene_cn,
    "seed": seed,
    "steps": steps_cn,
    "cfg_scale": cfg_cn,
    "canny": [canny_low, canny_high],
    "color_tone": custom_tone or color_tone,
    "timestamp": ts_now(),
}
img_path, meta_path = save_image_and_meta(cn_out, prefix="controlnet_canny", meta=meta, output_dir=CONFIG["OUTPUT_DIR"])
print("ControlNet done:", img_path)


In [ ]:
# @title ⬆️ Upscale x4
from scripts.loaders import get_upscale_pipe
from scripts.utils import save_image_and_meta, find_latest_image, ts_now
from scripts.logger import log_info
from scripts.config import VARIANT_MODELS
from PIL import Image
import os

UPSCALE_MODEL_ID = VARIANT_MODELS[CONFIG["MODEL_VARIANT"]]["upscale"]

up_input_path = ""  # путь или оставить пустым для последнего файла
if not up_input_path:
    up_input_path = find_latest_image(CONFIG["OUTPUT_DIR"])
    assert up_input_path, "No images found to upscale."

img = Image.open(up_input_path).convert("RGB")
pipe_up = get_upscale_pipe(UPSCALE_MODEL_ID, DEVICE, DTYPE)
upscaled = pipe_up(prompt="", image=img).images[0]

meta = {
    "model_variant": CONFIG["MODEL_VARIANT"],
    "mode": "upscale_x4",
    "input": os.path.basename(up_input_path),
    "timestamp": ts_now(),
}
up_path, meta_path = save_image_and_meta(upscaled, prefix="upscaled_x4", meta=meta, output_dir=CONFIG["OUTPUT_DIR"])

log_info(f"Upscaled: {up_path}")
print("Upscale done:", up_path)
